In [3]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import country_converter as coco

In [6]:
newdoc = doc.get_element_by_id("nav-today")
newdoc.xpath('//th') 

[<Element th at 0x7f2149094598>,
 <Element th at 0x7f2149094728>,
 <Element th at 0x7f2149094778>,
 <Element th at 0x7f21490948b8>,
 <Element th at 0x7f2149094908>,
 <Element th at 0x7f2149094958>,
 <Element th at 0x7f21490949a8>,
 <Element th at 0x7f2149094a98>,
 <Element th at 0x7f2149094b38>,
 <Element th at 0x7f2149094b88>,
 <Element th at 0x7f2149094bd8>,
 <Element th at 0x7f2149094c28>,
 <Element th at 0x7f2149094c78>,
 <Element th at 0x7f2149094cc8>,
 <Element th at 0x7f2149094d18>,
 <Element th at 0x7f2149094d68>,
 <Element th at 0x7f2149094db8>,
 <Element th at 0x7f2149094e08>]

### Build df from Url

In [37]:
url='https://www.worldometers.info/coronavirus/'

# Scraping the Url
page = requests.get(url)
doc = lh.fromstring(page.content)
todaydoc = doc.get_element_by_id("main_table_countries_today")

# Parse data
th_elements = todaydoc.xpath('//th') # header
td_elements = todaydoc.xpath('//td') # cells content

headers = [th_element.text_content() for th_element in th_elements]
content = [td_element.text_content() for td_element in td_elements]
rows_content = np.array(content).reshape(int(len(content)/len(headers)),len(headers)).tolist()

df = pd.DataFrame(rows_content)
df.columns = headers
df = df[:-1] # drop Total row

ValueError: cannot reshape array of size 3375 into shape (187,18)

In [43]:
content

['China',
 '81,008',
 '+41',
 '3,255                                ',
 '+7',
 '71,740',
 '6,013',
 '1,927',
 '56',
 'Italy',
 '47,021',
 '',
 '4,032                                ',
 '',
 '5,129',
 '37,860',
 '2,655',
 '778',
 'Spain',
 '24,926',
 '+3,355',
 '1,326                                ',
 '+233',
 '2,125',
 '21,475',
 '939',
 '533',
 'Germany',
 '20,705',
 '+857',
 '72                                ',
 '+4',
 '209',
 '20,424',
 '2',
 '247',
 'Iran',
 '20,610',
 '+966',
 '1,556                                ',
 '+123',
 '7,635',
 '11,419',
 '',
 '245',
 'USA',
 '19,775',
 '+392',
 '276                                ',
 '+20',
 '147',
 '19,352',
 '64',
 '60',
 'France',
 '12,612',
 '',
 '450                                ',
 '',
 '1,587',
 '10,575',
 '1,297',
 '193',
 'S. Korea',
 '8,799',
 '+147',
 '102                                ',
 '+8',
 '2,612',
 '6,085',
 '59',
 '172',
 'Switzerland',
 '6,186',
 '+571',
 '58                                ',
 '+2',
 '15',
 '6,1

In [35]:
print(len(lh.tostring(newdoc)))

206400


In [ ]:
root = html.parse("example.html").getroot()
element = root.get_element_by_id("hello")
print html.tostring(element)

In [52]:
# Convert values to float
for i,col_name in enumerate(df.columns):
    print
    if i!=0:
        df[col_name] = pd.to_numeric(df[col_name].apply(lambda x:x.replace(",","")),errors='coerce')
        
# Convert country to ISO codes
countries_list = df["Country,Other"]\
.apply(lambda x: x.strip()).replace({'UK': 'Great Britain', 'UAE': 'United Arab Emirates'}).values.tolist()
df["iso_alpha"] = pd.Series(coco.convert(names=countries_list, to='ISO3', not_found=None))

# Rename comma seperated cols
df = df.rename(columns={'Country,Other': 'Country',
                  'Serious,Critical': 'Critical'})

df = df.fillna(0)

# Create text that will be display on hover
df["text"] = df['Country'].apply(lambda x: x.strip()) + '<br>' + \
    'Active Cases ' + df['ActiveCases'].astype(int).astype(str) + \
    '<br>' + 'Total Deaths ' + df['TotalDeaths'].astype(int).astype(str)

In [53]:
df.shape

(353, 11)

In [9]:
# Export Dataframe
df.to_csv("static/data/corona.csv",index=False,sep=",")

### Visualize df using Plotly (Optional)

In [ ]:
# import plotly.express as px
# import datetime
# today_date = datetime.datetime.today().date().strftime("%d-%m-%Y")
# fig = px.choropleth(df, locations="iso_alpha",
#                     color="TotalCases",
#                     hover_name="Country",
#                     color_continuous_scale=px.colors.diverging.Portland,
#                    title='Daily Coronavirus Cases in the Word [{}]'.format(today_date)\
#                     +' Source: <a https://www.worldometers.info/coronavirus/">Worldometers</a>',
#                    height=600,
#                    range_color=[0,1000],
#                    labels={'TotalCases':'Min Number of cases'})
# fig.show()